In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [3]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [4]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [5]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [6]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data, _ = data.cuda(), Variable(_)
            recon, mu, log_var = vae(data)
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
for epoch in range(1, 50):
    train(epoch)
    test()

C:\Users\lyeoni\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 545.259648
Train Epoch: 1 [10000/60000 (17%)]	Loss: 187.443984
Train Epoch: 1 [20000/60000 (33%)]	Loss: 167.431328
Train Epoch: 1 [30000/60000 (50%)]	Loss: 173.935645
Train Epoch: 1 [40000/60000 (67%)]	Loss: 169.574434
Train Epoch: 1 [50000/60000 (83%)]	Loss: 167.623828
====> Epoch: 1 Average loss: 178.2732
====> Test set loss: 160.5496
Train Epoch: 2 [0/60000 (0%)]	Loss: 157.746084
Train Epoch: 2 [10000/60000 (17%)]	Loss: 163.131045
Train Epoch: 2 [20000/60000 (33%)]	Loss: 154.027910
Train Epoch: 2 [30000/60000 (50%)]	Loss: 150.750771
Train Epoch: 2 [40000/60000 (67%)]	Loss: 154.434961
Train Epoch: 2 [50000/60000 (83%)]	Loss: 160.234043
====> Epoch: 2 Average loss: 157.6318
====> Test set loss: 154.3878
Train Epoch: 3 [0/60000 (0%)]	Loss: 158.881689
Train Epoch: 3 [10000/60000 (17%)]	Loss: 147.508887
Train Epoch: 3 [20000/60000 (33%)]	Loss: 155.814082
Train Epoch: 3 [30000/60000 (50%)]	Loss: 149.681914
Train Epoch: 3 [40000/60000 (67%)]	Loss: 154.72

Train Epoch: 22 [40000/60000 (67%)]	Loss: 139.960703
Train Epoch: 22 [50000/60000 (83%)]	Loss: 132.134707
====> Epoch: 22 Average loss: 137.8886
====> Test set loss: 139.3573
Train Epoch: 23 [0/60000 (0%)]	Loss: 130.868770
Train Epoch: 23 [10000/60000 (17%)]	Loss: 137.820898
Train Epoch: 23 [20000/60000 (33%)]	Loss: 137.876045
Train Epoch: 23 [30000/60000 (50%)]	Loss: 136.780986
Train Epoch: 23 [40000/60000 (67%)]	Loss: 133.258955
Train Epoch: 23 [50000/60000 (83%)]	Loss: 136.125664
====> Epoch: 23 Average loss: 137.6203
====> Test set loss: 139.5251
Train Epoch: 24 [0/60000 (0%)]	Loss: 146.934863
Train Epoch: 24 [10000/60000 (17%)]	Loss: 137.225547
Train Epoch: 24 [20000/60000 (33%)]	Loss: 143.933516
Train Epoch: 24 [30000/60000 (50%)]	Loss: 137.208721
Train Epoch: 24 [40000/60000 (67%)]	Loss: 138.820586
Train Epoch: 24 [50000/60000 (83%)]	Loss: 145.561338
====> Epoch: 24 Average loss: 137.5135
====> Test set loss: 139.3198
Train Epoch: 25 [0/60000 (0%)]	Loss: 139.099023
Train Epoch: 

====> Test set loss: 137.8069
Train Epoch: 44 [0/60000 (0%)]	Loss: 130.099238
Train Epoch: 44 [10000/60000 (17%)]	Loss: 138.025166
Train Epoch: 44 [20000/60000 (33%)]	Loss: 135.973965
Train Epoch: 44 [30000/60000 (50%)]	Loss: 136.995664
Train Epoch: 44 [40000/60000 (67%)]	Loss: 136.900977
Train Epoch: 44 [50000/60000 (83%)]	Loss: 136.051191
====> Epoch: 44 Average loss: 134.9452
====> Test set loss: 137.4019
Train Epoch: 45 [0/60000 (0%)]	Loss: 127.148193
Train Epoch: 45 [10000/60000 (17%)]	Loss: 137.502168
Train Epoch: 45 [20000/60000 (33%)]	Loss: 129.090859
Train Epoch: 45 [30000/60000 (50%)]	Loss: 132.195811
Train Epoch: 45 [40000/60000 (67%)]	Loss: 135.177314
Train Epoch: 45 [50000/60000 (83%)]	Loss: 138.083076
====> Epoch: 45 Average loss: 134.6897
====> Test set loss: 137.5036
Train Epoch: 46 [0/60000 (0%)]	Loss: 129.757295
Train Epoch: 46 [10000/60000 (17%)]	Loss: 131.129844
Train Epoch: 46 [20000/60000 (33%)]	Loss: 131.382236
Train Epoch: 46 [30000/60000 (50%)]	Loss: 133.167402

In [8]:
from torchvision.utils import save_image
with torch.no_grad():
    sample = torch.randn(64, 2).cuda()
    sample = vae.decoder(sample).cuda()
    save_image(sample.view(64, 1, 28, 28), 'sample_' + '.png')

C:\Users\lyeoni\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
